In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook


In [2]:
import os

import graph_tool.all as gt
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')
# Style
sns.set_theme(context='talk', style='white', palette='Set2')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


# Meta and Parameters

In [3]:
# Load metadata
meta = get_meta()

# Subject preview
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS information available
        # assert row['BRAAK_AD'] in (3, 4, 5)
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}')
filtered = np.sort(filtered)
for i in range(len(filtered)):
    print(filtered[i])

# Parameters
print(f'\nAvailable attention columns: {get_attention_columns()}')
column_ad = get_attention_columns()[0]
column_scz = get_attention_columns()[2]
column_data = get_attention_columns()[4]
synthetic_nodes_of_interest = ['OPC', 'Micro', 'Oligo', 'EN']


/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/file.py:108: DtypeWarning: Columns (5,157,158,159,161,164,165,166,167,170,172,176,177,178,179,189,191,193,195,276,285,286,287,288,289,290,291,292,293,294,295,297,342,345,346,462,465,566,570,571,572,573,574,580,582,583,584,586,589,592,594,597,599,601,603,606,607,611,613,615,617,618,620,622,625,684,686) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(META)



Available attention columns: ['att_D_AD_1', 'att_D_AD_2', 'att_D_SCZ_1', 'att_D_SCZ_2', 'att_D_no_prior_0', 'att_D_no_prior_1', 'att_D_no_prior_2', 'att_D_no_prior_3']


# Plots

## Individual Comparisons (Figure 3)

In [5]:
individual_comparisons = [
    # M19050 Hispanic Female, 74.0, BRAAK 5.0
    # M59593 Hispanic Female, 76.0, BRAAK 5.0
    # M72079 Black Female, 64.0, BRAAK 6.0
    # M41496 Black Female, 76.0, BRAAK 4.0
    # M11589 Black Female, 63.0, BRAAK 2.0
    # M73342 Black Female, 62.0, BRAAK 0.0
    # (subject_id_1, subject_id_2, column)
    # for subject_id_1, subject_id_2, column in individual_comparisons:
    ('M19050', 'M59593', column_ad),  # AD - AD
    # ('M72079', 'M41496', column_ad),  # AD - High BRAAK
    # ('M72079', 'M11589', column_ad),  # AD - Low BRAAK
    # ('M72079', 'M73342', column_ad),  # AD - CTRL
]
palette = plt.rcParams['axes.prop_cycle'].by_key()['color']
individual_colors = {
    sid: rgba_to_hex(palette[i]) for i, sid in enumerate(
        sum([list(comparison[:2]) for comparison in individual_comparisons], []))
}

# Verify all are available
for subject_id_1, subject_id_2, column in individual_comparisons:
    for sid in [subject_id_1, subject_id_2]:
        load_graph_by_id(sid)


### 3A Mini Plots

In [6]:
for i, (subject_id_1, subject_id_2, column) in enumerate(individual_comparisons):
    print(' - '.join((subject_id_1, subject_id_2, column)))

    # Assemble
    sids = [subject_id_1, subject_id_2]
    gs = [compute_graph(load_graph_by_id(sid, column=column)) for sid in sids]

    # Filter
    gs = [
        filter_to_synthetic_vertices(g.copy(), vertex_ids=synthetic_nodes_of_interest)
        for g in gs
    ]

    # Recalculate
    gs = [assign_vertex_properties(g) for g in gs]

    # Plot
    fig, axs = get_mosaic([list(range(2))], scale=9)
    plot_graph_comparison(gs, axs=axs, subject_ids=sids)
    fig.savefig(f'../plots/individual_mini_{"-".join(sids)}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
    print()


M19050 - M59593 - att_D_AD_1
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████████████| 649/649 [00:00<00:00, 217942.62it/s]

Calculating positions...


Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████████████| 344/344 [00:00<00:00, 167538.39it/s]


...

Calculating positions...


Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████████████| 332/332 [00:00<00:00, 271497.16it/s]


Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████████████| 353/353 [00:00<00:00, 235687.57it/s]


### 3B Attention Comparisons

In [24]:
for subject_id_1, subject_id_2, _ in individual_comparisons:
    for column in get_attention_columns():
        print(' - '.join((subject_id_1, subject_id_2, column)))

        # Assemble
        sample_ids = [subject_id_1, subject_id_2]
        graphs = [compute_graph(load_graph_by_id(sid, column=column)) for sid in sample_ids]

        # Get graph
        g = concatenate_graphs(*graphs, threshold=False)
        g = get_intersection(g)
        g = cull_isolated_leaves(g)

        fig, axs = get_mosaic([list(range(1))], scale=6)
        df = plot_individual_edge_comparison(g, sample_ids, ax=axs[0])
        plt.tight_layout()
        fig.savefig(f'../plots/individual_edge_comparison_{"-".join((subject_id_1, subject_id_2))}_{column}.pdf', format='pdf', transparent=True, backend='cairo')
        print()


M19050 - M59593 - att_D_AD_1
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████| 3347/3347 [00:00<00:00, 240090.57it/s]
/tmp/ipykernel_700/3732660470.py:16: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



M19050 - M59593 - att_D_AD_2
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████| 3347/3347 [00:00<00:00, 228120.96it/s]
/tmp/ipykernel_700/3732660470.py:16: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



M19050 - M59593 - att_D_SCZ_1
Removing duplicate edges...


...


M72079 - M73342 - att_D_no_prior_2
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████| 3797/3797 [00:00<00:00, 302122.29it/s]
/tmp/ipykernel_700/3732660470.py:16: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



M72079 - M73342 - att_D_no_prior_3
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████████| 3797/3797 [00:00<00:00, 201237.98it/s]


/tmp/ipykernel_700/3732660470.py:16: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### 3C Pathway Enrichment (MANUAL)

In [8]:
# Enrichment
for subject_id_1, subject_id_2, column in individual_comparisons:
    # Assemble
    sample_ids = [subject_id_1, subject_id_2]
    graphs = [load_graph_by_id(sid, column=column) for sid in sample_ids]
    for i in range(len(graphs)):
        # Remove self-loops
        graphs[i] = graphs[i].loc[graphs[i].apply(lambda x: x['TF'] != x['TG'], axis=1)]

        # Set index, combine tf and tg, rename coef column
        graphs[i].index = graphs[i].apply(lambda x: get_edge_string([x['TF'], x['TG']]), axis=1)
        graphs[i] = graphs[i].drop(columns=['TF', 'TG'])
        graphs[i] = graphs[i].rename(columns={'coef': sample_ids[i]})

        # Drop duplicates
        # TODO: Why are there duplicates?
        graphs[i] = graphs[i].drop_duplicates()

    # Filter to common graphs and join
    graphs = graphs[0].join(graphs[1], how='inner')

    # Get differentially expressed genes
    graphs['Difference'] = np.abs(graphs[subject_id_1] - graphs[subject_id_2])
    graphs = graphs.sort_values('Difference', ascending=False)

    # Write to file
    fname_prefix = f'../plots/individual_pathway_enrichment_{"-".join((subject_id_1, subject_id_2))}_{column}'
    f_tfs = open(fname_prefix + '.tfs.txt', 'w')
    f_tgs = open(fname_prefix + '.tgs.txt', 'w')
    f_edges = open(fname_prefix + '.edges.txt', 'w')
    for edge in graphs.index:
        # TODO: Only show unique tf, tg, maybe?
        f_edges.write(edge + '\t' + f'{graphs.loc[edge, "Difference"]:.5f}' + '\n')
        tf, tg = edge.split(get_edge_string())
        # Don't show synthetic nodes
        if not string_is_synthetic(tf): f_tfs.write(tf + '\n')
        if not string_is_synthetic(tg): f_tgs.write(tg + '\n')
    f_tfs.close()
    f_tgs.close()
    f_edges.close()

    # MANUAL PROCESSING
    # Run the output from '<fname_prefix>.xxs.txt' in DisGeNet, save file
    # from Enrichment_QC/GO_DisGeNET as '<fname_prefix>.xxs.csv'

    # Plot enrichments
    get_enrichment_file = lambda x: f'{fname_prefix}.{x}'
    for ftype in ['tfs', 'tgs']:
        # Get file name
        fname = get_enrichment_file(ftype)

        # Get enrichment
        enrichment = get_enrichment(fname)
        if enrichment is None: continue

        # Plot
        fig, axs = get_mosaic([list(range(1))], scale=9)
        plot_enrichment(enrichment, ax=axs[0])
        fig.savefig(fname + '.pdf', format='pdf', transparent=True, backend='cairo')

        # Plot
        fig, axs = get_mosaic([list(range(1))], scale=9)
        plot_enrichment(enrichment, ax=axs[0])
        fig.savefig(fname + '.pdf', format='pdf', transparent=True, backend='cairo')


### 3D Attention Heatmap

In [9]:
# Parameters
column = column_ad
subject_ids = meta['SubID'].to_numpy()

# Load graphs
graphs, subject_ids = load_many_graphs(subject_ids, column=get_attention_columns())
# graphs = [compute_graph(g) for g in graphs]

# # Get attentions
# df = {}
# for column in get_attention_columns():
#     attention, _ = compute_edge_summary(graphs, subject_ids=subject_ids)
#     attention = attention.set_index('Edge')
#     df[column] = attention.var(axis=1)


In [10]:
# Set indices to edges and clean
print('Fixing indices...')
for i in tqdm(range(len(graphs))):
    graphs[i].index = graphs[i].apply(lambda r: get_edge_string([r['TF'], r['TG']]), axis=1)
    graphs[i] = graphs[i].drop(columns=['TF', 'TG'])
    # Remove duplicates
    graphs[i] = graphs[i][~graphs[i].index.duplicated(keep='first')]

# Get all unique edges
print('Getting unique edges...')
all_edges = np.unique(sum([list(g.index) for g in graphs], []))


# Standardize index order
print('Standardizing indices...')
for i in tqdm(range(len(graphs))):
    # Add missing indices and order based on `all_edges`
    # to_add = [edge for edge in all_edges if edge not in list(graphs[i].index)]  # SLOW
    to_add = list(set(all_edges) - set(graphs[i].index))

    # Empty rows
    new_rows = pd.DataFrame(
        [[np.nan]*len(graphs[i].columns)]*len(to_add),
        columns=graphs[i].columns,
    ).set_index(pd.Series(to_add))
    # Native concat
    graphs[i] = pd.concat([graphs[i], new_rows]).loc[all_edges]

# Convert to numpy
graphs = [g.to_numpy() for g in graphs]
attention_stack = np.stack(graphs, axis=-1)
# attention_stack.shape = (Edge, Head, Subject)
# attention_stack.shape = (all_edges, get_attention_columns(), subject_ids)


Fixing indices...


  0%|                                                                                           | 0/616 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 616/616 [00:04<00:00, 136.45it/s]


Getting unique edges...
Standardizing indices...


100%|█████████████████████████████████████████████████████████████████████████████████| 616/616 [06:06<00:00,  1.68it/s]


In [32]:
# Calculate heatmap
variance = np.var(np.nan_to_num(attention_stack), axis=2)
df = pd.DataFrame(variance, index=all_edges, columns=get_attention_columns())


In [33]:


fig, axs = get_mosaic([[0,0,0,0]]*6, scale=3)
sns.heatmap(data=df, ax=axs[0])  # , norm=LogNorm()
fig.savefig(f'../plots/individual_edge_variance_heatmap.pdf', format='pdf', transparent=True, backend='cairo')
plt.show()


### 3E Dosage Analysis

In [22]:
# Parameters
column = column_ad
subject_ids = meta['SubID'].to_numpy()

# Load graphs
graphs, subject_ids = load_many_graphs(subject_ids, column=column)
graphs = [compute_graph(g) for g in graphs]

# Get dosage information
dosage = get_dosage()
dosage = convert_dosage_ids_to_subject_ids(dosage, meta=meta)

# Get attention
attention, _ = compute_edge_summary(graphs, subject_ids=subject_ids)
attention = attention.set_index('Edge')


/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/file.py:112: DtypeWarning: Columns (20,24,25,57,59,61,90,145,231,249,270,271,286,292,297,318,355,397,437,510,511,627,632,646,662,702,732,738,761,786,804,805,833,837,868,891,913,918,924,953,954,955,956,957,958) have mixed types. Specify dtype option on import or set low_memory=False.
  dosage = pd.read_csv(DOSAGE)
/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/utility.py:1173: UserWarning: Unable to find SNP '8321' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')
/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/utility.py:1173: UserWarning: Unable to find SNP '38162' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')
/mnt/c/Users/nck/repos/GNN-Plus/Attention Analysis/scripts/functions/utility.py:1173: UserWarning: Unable to find SNP '208768' in metadata
  warnings.warn(f'Unable to find SNP \'{dosage_id}\' in metadata')
/

No threshold provided, using threshold of 0.01381273008154222.
Removing duplicate edges...


100%|█████████████████████████████████████████████████████████████████████| 1005000/1005000 [00:03<00:00, 302670.25it/s]


Filtered from 5047 vertices and 362559 edges to 1629 vertices and 12719 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 12719/12719 [00:02<00:00, 5782.52it/s]


In [27]:
list(meta.columns)


['SubID',
 'Brain_bank',
 'Age',
 'Sex',
 'Ethnicity',
 'Education',
 'PMI',
 'Dx',
 'pH',
 'CERAD',
 'BRAAK_AD',
 'PLAQUE',
 'BB',
 'SubNum',
 'Batch1_Barcode',
 'Batch1_Storage.Type',
 'Batch1_Brain.Region',
 'Batch1_UoM',
 'Batch1_Size',
 'Batch2_Barcode',
 'Batch2_Storage.Type',
 'Batch2_Brain.Region',
 'Batch2_UoM',
 'Batch2_Size',
 'Batch2_Loc.Type',
 'Batch2_Loc',
 'Batch2_Shell',
 'Batch2_Box',
 'CDRScore',
 'CERAD_1',
 'CERAD_2',
 'Plq_Mn',
 'HippoPlaquesValue',
 'EntorPlaquesValue',
 'AmygPlaquesValue',
 'HippoPlaquesWCoresValue',
 'EntorPlaquesWCoresValue',
 'AmygPlaquesWCoresValue',
 'HippoTanglesValue',
 'EntorTanglesValue',
 'AmygTanglesValue',
 'HippoLewyValue',
 'EntorLewyValue',
 'AmygLewyValue',
 'MidPlaquesValue',
 'SupPlaquesValue',
 'InfPlaquesValue',
 'OcciPlaquesValue',
 'MidPlaquesWCoresValue',
 'SupPlaquesWCoresValue',
 'InfPlaquesWCoresValue',
 'OcciPlaquesWCoresValue',
 'MidGliosisValue',
 'SupGliosisValue',
 'InfGliosisValue',
 'OcciGliosisValue',
 'MidLewyV

In [23]:
# Select target SNP
target_snp = dosage.index[42]  # Random for now

# Make df
data_dosage = dosage.loc[[target_snp]].T
data_attention = attention.T
df = data_dosage.join(data_attention, how='inner')

# Select target edge
p_min = 1
for edge in attention.index:
    corr, pval = scipy.stats.pearsonr(
        df[[edge]].to_numpy().squeeze(),
        df[[target_snp]].to_numpy().squeeze())
    if pval < p_min:
        p_min = pval
        best_corr = corr
        target_edge = edge
print(f'Found minimal p-value of {p_min:.6f} (Correlation: {best_corr:.6f}).')

# Format df
axis_snp = f'{target_snp} Dosage'
axis_edge = f'{target_edge} Attention'
df = df.rename(columns={target_snp: axis_snp, target_edge: axis_edge})

# Scatter
fig, axs = get_mosaic([list(range(1))], scale=9)
sns.scatterplot(data=df, x=axis_snp, y=axis_edge, ax=axs[0])
fig.savefig(f'../plots/individual_dosage_correlation_{column}.pdf', format='pdf', transparent=True, backend='cairo')


Found minimal p-value of 0.000171 (Correlation: 0.163327).


## Group Comparisons (Figure 4)

In [40]:
# Combinations
# TODO: Potentially move each entry to dictionary, so changes in order
#   are easier to propagate
contrast_groupings = [
    # (contrast name, contrast group, attention column, comparison column, target meta column, other target meta column)
    # for contrast_name, contrast_group, column, comparison, target, target_comparison in contrast_groupings:
    # TODO: Revise ethnicity prediction
    ('c15x', 'AD', column_data, column_ad, 'Ethnicity', 'BRAAK_AD'),
    # ('c06x', 'AD', column_ad, column_data, 'BRAAK_AD'),  # Eventually SCZ, BP and such
    # ('c71x', 'MoodDys', column_data, column_ad, 'nps_MoodDysCurValue'),  # Dysphoria
    # ('c72x', 'DecInt', column_data, column_ad, 'nps_DecIntCurValue'),  # Anhedonia
]


### 4A Variance Heatmap

In [35]:
# Get plots for each column
for contrast_name, _, column, comparison, _, _ in contrast_groupings:
    for col in (column, comparison):
        print(' - '.join((contrast_name, col)))

        # Get contrast
        contrast = get_contrast(contrast_name)

        # Compute
        df_subgroup = compute_contrast_summary(contrast, column=col)

        # Plot mean-sorted
        fig, axs = get_mosaic([list(range(1))], scale=9)
        plot_subgroup_heatmap(df_subgroup, ax=axs[0])
        plt.tight_layout()
        fig.savefig(f'../plots/group_variance_heatmap_{contrast_name}_{col}.pdf', format='pdf', transparent=True, backend='cairo')


c15x - att_D_no_prior_0
No threshold provided, using threshold of 0.017553121661133885.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 698279/698279 [00:02<00:00, 309781.98it/s]


Filtered from 5030 vertices and 270687 edges to 1463 vertices and 10173 edges via common edge filtering.
No threshold provided, using threshold of 0.04979338607285737.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 146728/146728 [00:00<00:00, 312538.86it/s]


Filtered from 4719 vertices and 81268 edges to 684 vertices and 3350 edges via common edge filtering.


...

100%|████████████████████████████████████████████████████████████████████████████| 3350/3350 [00:00<00:00, 24357.50it/s]


No threshold provided, using threshold of 0.01546432984072714.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 845007/845007 [00:02<00:00, 318097.24it/s]


Filtered from 5039 vertices and 315575 edges to 1488 vertices and 10734 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 10734/10734 [00:01<00:00, 6827.90it/s]
/tmp/ipykernel_700/1238480732.py:15: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### 4B Distribution Comparison

In [39]:
# TODO: Use highest variance rather than random edges
# TODO: Make both columns same scale, maybe?
for contrast_name, _, column, comparison, target, _ in contrast_groupings:
    print(' - '.join((contrast_name, column, target)))
    # Get contrast
    contrast = get_contrast(contrast_name)

    # Plot
    fig, axs = get_mosaic([2*[0], 2*[1]], scale=3)
    sns.despine()

    _, edges_include = plot_BRAAK_comparison(
        contrast,
        # {k: v[:10] for k, v in contrast.items()},
        meta=meta,
        column=column,
        target=target,
        legend=False,
        ax=axs[0])
    plt.xlabel(None)
    plt.ylabel(column)
    plt.xticks([])

    plot_BRAAK_comparison(
        contrast,
        meta=meta,
        column=comparison,
        target=target,
        edges_include=edges_include,
        ax=axs[1])
    plt.ylabel(comparison)

    fig.savefig(f'../plots/group_differential_expression_{contrast_name}_{column}_{comparison}_{target}.pdf', format='pdf', transparent=True, backend='cairo')
    print()


c15x - att_D_no_prior_0 - Ethnicity
No threshold provided, using threshold of 0.01546432984072714.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 845007/845007 [00:02<00:00, 327763.05it/s]


Filtered from 5039 vertices and 315575 edges to 1488 vertices and 10734 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 10734/10734 [00:01<00:00, 7225.57it/s]


No threshold provided, using threshold of 0.01546432984072714.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 845007/845007 [00:03<00:00, 248182.11it/s]


Filtered from 5039 vertices and 315575 edges to 1488 vertices and 10734 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 10734/10734 [00:01<00:00, 7209.93it/s]


### 4C Cross-Validation Accuracies

In [41]:
# TODO: Make all y-labels horizontal
for contrast_name, _, column, _, target, target_comparison in contrast_groupings:
    for tar in (target, target_comparison):
        # if contrast_name != 'c71x': continue
        print(' - '.join((contrast_name, column, tar)))
        # Get contrast
        contrast = get_contrast(contrast_name)

        # Compute prioritized edges
        # Get 100 most variant edges
        # TODO: Revise this method, maybe also consider means
        sids = sum([sids for _, sids in contrast.items()], [])
        df_subgroup = compute_contrast_summary(contrast, column=column)
        df = join_df_subgroup(df_subgroup, num_sort=100)
        prioritized_edges = list(df.index)

        # Plot
        # TODO: Maybe return to row-normalization
        fig, axs = get_mosaic([[0]], scale=9)
        df, acc = plot_prediction_confusion(contrast, meta=meta, column=column, target=tar, prioritized_edges=prioritized_edges, classifier_type='SGD', ax=axs[0])

        # Save plot
        fname_prefix = f'../plots/group_prioritized_edge_prediction_{contrast_name}_{column}_{tar}'
        fig.savefig(f'{fname_prefix}.pdf', format='pdf', transparent=True, backend='cairo')

        # Save text
        f_edges = open(f'{fname_prefix}.edges.txt', 'w')
        f_tfs = open(f'{fname_prefix}.tfs.txt', 'w')
        f_tgs = open(f'{fname_prefix}.tgs.txt', 'w')
        for edge in prioritized_edges:
            f_edges.write(edge + '\n')
            tf, tg = edge.split(get_edge_string(['', '']))
            f_tfs.write(tf + '\n')
            f_tgs.write(tg + '\n')
        f_edges.close()
        f_tfs.close()
        f_tgs.close()

        # CLI
        print()


c15x - att_D_no_prior_0 - Ethnicity
No threshold provided, using threshold of 0.017553121661133885.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 698279/698279 [00:02<00:00, 317518.60it/s]


Filtered from 5030 vertices and 270687 edges to 1463 vertices and 10173 edges via common edge filtering.
No threshold provided, using threshold of 0.04979338607285737.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 146728/146728 [00:00<00:00, 307304.66it/s]


Filtered from 4719 vertices and 81268 edges to 684 vertices and 3350 edges via common edge filtering.


...

100%|███████████████████████████████████████████████████████████████████████████| 10734/10734 [00:01<00:00, 6633.76it/s]


No threshold provided, using threshold of 0.01546432984072714.
Removing duplicate edges...


100%|███████████████████████████████████████████████████████████████████████| 845007/845007 [00:02<00:00, 316957.47it/s]


Filtered from 5039 vertices and 315575 edges to 1488 vertices and 10734 edges via common edge filtering.


100%|███████████████████████████████████████████████████████████████████████████| 10734/10734 [00:01<00:00, 6880.93it/s]


## General

### Graph Legend

In [38]:
# Plot legend
plt.clf()
plot_legend()
plt.gca().axis('off')
plt.tight_layout()
plt.savefig(f'../plots/graph_legend.pdf', format='pdf', transparent=True, backend='cairo')


/tmp/ipykernel_700/3570599204.py:5: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
